In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ANNUAL_DISCOUNT_RATE = 0.15
CONST_RETENTION_RATE = 0.56530017



$$(r_{mon}+1)^{12} = 1+r_{year}$$

$$r_{mon} = \sqrt[12]{1+r_{year}}-1$$

In [8]:
MONTHLY_DISCOUNT_RATE = (np.power(1+ANNUAL_DISCOUNT_RATE, 1/12) - 1)
print(MONTHLY_DISCOUNT_RATE)

0.01171491691985338


# 6a

Our data is from the UK from 2018. According to [FourWeekMBA](https://fourweekmba.com/netflix-average-monthly-revenue-per-subscriber/) and their figures from Netflix' financials, the EMEA (Europe, Middle-East and Africa) region had Average Monthly Revenue per Subscriber in 2018 of $10.45, which seems reasonable compared to today's monthly rates between $6.99 - $22.99 and optional add-ons.  

We will use the reported average revenue per user of **$10.45** and assume that this is the revenue per paying user per month

We will now calculate estimates of future revenues per customer, and this could either be done assuming a customer that stays for a whole year, or using the survival probabilities. We will here try to calculate using the constant retention rate as we want to calculate the future revenues of all customers in the customer base and not just the ones we want to acquire now, and get a more general result

In [9]:
from main import get_all_retention_rates_time_normalized, get_data

data = get_data()

get_all_retention_rates_months_after_acquisition = get_all_retention_rates_time_normalized(data)

avg_monthly_retention_rate_across_cohorts = np.apply_along_axis(np.nanmean,axis = 0, arr = get_all_retention_rates_months_after_acquisition)

active_probs_changing_retention = avg_monthly_retention_rate_across_cohorts[1:-1]
active_probs_changing_retention = np.insert(active_probs_changing_retention,0,1)
active_probs_changing_retention = np.cumprod(active_probs_changing_retention)
active_probs_constant_retention = [CONST_RETENTION_RATE**i for i in range(12)]


ARPU = 10.45

cash_flow = [round(ARPU*active_probs_constant_retention[i]/np.power(1+MONTHLY_DISCOUNT_RATE, i),5) for i in range(12)]
print(cash_flow)
CLV_year = sum(cash_flow)
print(CLV_year)

[10.45, 5.83898, 3.26256, 1.82297, 1.01859, 0.56914, 0.31801, 0.17769, 0.09929, 0.05548, 0.031, 0.01732]
23.66103


/opt/homebrew/lib/python3.10/site-packages/numpy/lib/shape_base.py:379: RuntimeWarning: Mean of empty slice
  res = asanyarray(func1d(inarr_view[ind0], *args, **kwargs))
/opt/homebrew/lib/python3.10/site-packages/numpy/lib/shape_base.py:402: RuntimeWarning: Mean of empty slice
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))


# 6b

Assuming zero marginal costs of an extra customer is very reasonable, as the service is already there and serves many customers. Servicing more customers does cost more in terms of server capacity, but this could be seen as fixed costs up to a given capacity, because servicing an extra customer will in most cases not require an increase in capacity

# 6c

We assume 0 marginal costs for a customer, and therefore consider all revenue as profit. From an accounting perspective we will in this regard treat all Netflix' costs as fixed

We are also assuming that we are now at the beginning of month 12, and want to calculate the CLV for acquiring a customer right now

In [10]:
from mpmath import nsum, inf

AVG_RETENTION_RATE_COHORT_0 = 0.178726

def survival_prob_for_month_const_rate(month: int):
    return np.power(CONST_RETENTION_RATE, month)

def survival_prob_for_month_cobort_0(month: int):
    return np.power(AVG_RETENTION_RATE_COHORT_0, month)

survival_probs_const_rate = [survival_prob_for_month_cobort_0(i) for i in range(0, 13)]
print(survival_probs_const_rate)

print(nsum(lambda n: ARPU*survival_prob_for_month_cobort_0(n)/(1+MONTHLY_DISCOUNT_RATE)**n, [0, inf]))

[1.0, 0.178726, 0.031942983076, 0.005709041593241176, 0.0010203541677936223, 0.00018236381899308293, 3.259315591335774e-05, 5.825244383770776e-06, 1.0411226277338157e-06, 1.8607568276435392e-07, 3.325656247774192e-08, 5.943812385396902e-09, 1.0623138123924467e-09]
12.6921507202114


# 6d

In [11]:

constant_rates_expected_lifetime = 2.300437982687964
changing_rates_expected_lifetime = 1.505681818203322

print(ARPU*constant_rates_expected_lifetime)
print(ARPU*changing_rates_expected_lifetime)



24.03957691908922
15.734375000224714


In [12]:
#The apporach above does not seem to take into account the discount rate
#I propose the following alternative solution
#The difference is small though, because of the short time horizon and relatively low monthly discount rate

discount_factor_array = [1/(1+MONTHLY_DISCOUNT_RATE)**n for n in range(12)]


def clv_from_expectation_value(expected_lifetime):
    lifetime_left = expected_lifetime
    counter = 0 
    clv = 0
    while lifetime_left >=1:
        clv += ARPU*discount_factor_array[counter]
        counter += 1
        lifetime_left -=1
    clv += lifetime_left*ARPU*discount_factor_array[counter]
    return clv
print(clv_from_expectation_value(constant_rates_expected_lifetime))
print(clv_from_expectation_value(changing_rates_expected_lifetime))

23.84628653658242
15.673185812375776


# 6e

For [6d](#6d) with the expected lifetime method we see higher CLV's for both constant and changing retention rates in comparison to using an infinite time horizon and the survival probabilities for each month. For [6c](#6c) we only use the average constant retention rate for cohort 0 and not the average across all cohorts. The average retention rate for cohort 0 is lower than the average of all cohorts. In addition to discounting the cash flow. The biggest difference is however accounting for cohort 0 and not averaging across all cohorts, since cohort 0 was really unprofitable compared to the other cohorts for last year. If we would have used the average retention rate across all cohorts, we would get almost the same expected CLV. In summary cohort 0, which is the month we are in now, was a much less profitable group of acquired users last year then the other cohorts, and we expect a lower CLV of $\$12.69$ for this cohort.